## Пишем классификатор: попытка №2

In [7]:
from nltk.corpus import stopwords
import csv
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.metrics import confusion_matrix
import gensim
import numpy as np
from sklearn.linear_model import LogisticRegression

In [8]:
df_res2 = pd.read_csv('df_res2_modified.csv')

/tmp/ipykernel_798/710896861.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res2 = pd.read_csv('df_res2_modified.csv')


In [9]:
df_res2 = df_res2[['id', 'abstract', 'check']]
df_res2['abstract'] = df_res2['abstract'].fillna('')

/tmp/ipykernel_798/1053174206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res2['abstract'] = df_res2['abstract'].fillna('')


Теперь векторизируем через скаченную предобученную модель FastText

In [10]:
modell = gensim.models.KeyedVectors.load_word2vec_format('cc.en.300.vec.gz', binary=False)

In [11]:
def text_to_vector(text):
    words = text.split()
    vectors = [modell[word] for word in words if word in modell]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(modell.vector_size)

In [12]:
data = df_res2["abstract"]
y = df_res2["check"]

In [13]:
X = np.array([(text, text_to_vector(text)) for text in data])
y = np.array(y)

/tmp/ipykernel_798/1492243799.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([(text, text_to_vector(text)) for text in data])


Прогоним через модель логистической регрессии

In [14]:
tuple_train, tuple_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
sentence_train = tuple_train[:, 0]
vector_train = tuple_train[:, 1]
sentence_test = tuple_test[:, 0]
vector_test = tuple_test[:,1]

In [16]:
X_train = np.stack(vector_train)
X_test = np.stack(vector_test)

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

/home/anna_mikhneva_/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
model.predict(X_test)[:5]

array([1, 0, 1, 1, 1])

In [18]:
y_pred = model.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score, f1_score
#accuracy_score(y_test, y_pred)

In [15]:
f1_score(y_test, y_pred)

0.9625426672255823

In [16]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[16267   329]
 [ 2173 32147]]


In [19]:
false_positive_indices = sentence_test[(y_pred == 1) & (y_test == 0)] #не удалил, модель сказала удалить
false_negative_indices = sentence_test[(y_pred == 0) & (y_test == 1)] #удалил, а модель не удалила

In [21]:
false_positive_examples = pd.DataFrame(false_positive_indices, columns=['Values'])
false_negative_examples = pd.DataFrame(false_negative_indices, columns=['Values'])
print("False Positive Examples:")
print(false_positive_examples[:20])

print("\nFalse Negative Examples:")
print(false_negative_examples[:20])

False Positive Examples:
                                               Values
0   Hittite grammar has been of great interest to ...
1   An abstract is not available for this content ...
2   We’ve built this digital research platform to ...
3   Dr. I. M. Nick enjoys a well-deserved reputati...
4   This study examines the syntactical difference...
5   IN THE SPRING OF 1954, Vol. I, NO.1 of Names i...
6   We’ve built this digital research platform to ...
7   We’ve built this digital research platform to ...
8   Review article on Grammar of Mandarin . John B...
9                                      Editorial Note
10  Review of Yanez-Bouza, Nuria. Grammar, Rhetori...
11  We’ve built this digital research platform to ...
12  Preview this article: Response to Traugott, Pa...
13  With this Report (here the n° 2) we present re...
14  Article Klaus Zimmermann/Birte Kellermeier-Reh...
15                               Short communications
16  high tone low tone mid tone falling (high to l...
17 

In [22]:
false_positive_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])
false_negative_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])

In [23]:
#for example in false_positive_examples.values:
    #false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)
#for example in false_negative_examples.values:
    #false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)

In [22]:
false_positive_df.to_csv('false_positive_df2_modified.csv', index=False)
false_negative_df.to_csv('false_negative_df2_modified.csv', index=False)

Прогоним через модель дерева решений

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
model_tree = DecisionTreeClassifier(max_depth=7)

model_tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7)

In [26]:
y_pred = model_tree.predict(X_test)

In [26]:
accuracy_score(y_test, y_pred)

0.9500157121533506

In [27]:
f1_score(y_test, y_pred)

0.9618743726873699

In [28]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[16267   329]
 [ 2216 32104]]


In [27]:
false_positive_indices = sentence_test[(y_pred == 1) & (y_test == 0)] #не удалил, модель сказала удалить
false_negative_indices = sentence_test[(y_pred == 0) & (y_test == 1)] #удалил, а модель не удалила

In [29]:
false_positive_examples = pd.DataFrame(false_positive_indices, columns=['Values'])
false_negative_examples = pd.DataFrame(false_negative_indices, columns=['Values'])
print("False Positive Examples:")
print(false_positive_examples[:20])

print("\nFalse Negative Examples:")
print(false_negative_examples[:20])

False Positive Examples:
                                               Values
0   An abstract is not available for this content ...
1   We’ve built this digital research platform to ...
2   Dr. I. M. Nick enjoys a well-deserved reputati...
3   Towards target-like language use: Russian and ...
4   Chickasaw is a Muskogean language spoken in so...
5   We’ve built this digital research platform to ...
6   Spanish is one of the most spoken languages in...
7   We’ve built this digital research platform to ...
8   Abstract As a phenomenon arising from the inte...
9   Review article on Grammar of Mandarin . John B...
10  This article is focused on the Latin verb uert...
11  The article is an in-depth review of Petar Keh...
12  We’ve built this digital research platform to ...
13  Preview this article: Response to Traugott, Pa...
14  With this Report (here the n° 2) we present re...
15  Article Klaus Zimmermann/Birte Kellermeier-Reh...
16  Individual and collective memories are semioti...
17 

In [30]:
false_positive_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])
false_negative_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])

# for example in false_positive_examples.values:
#     false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)
# for example in false_negative_examples.values:
#     false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)

In [32]:
false_positive_df.to_csv('false_positive_tree_modified.csv', index=False)
false_negative_df.to_csv('false_negative_tree_modified.csv', index=False)

Прогоним через модель случайного леса

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
model_forest = RandomForestClassifier(
    n_estimators=50, 
    max_depth=5,
    max_samples=0.2,
    max_features=7,
    random_state=0, 
)
# путём перебора разных значений параметров, выяснили, что с такими числами работает лучше всего

model_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=7, max_samples=0.2,
                       n_estimators=50, random_state=0)

In [33]:
y_pred = model_forest.predict(X_test)

In [36]:
accuracy_score(y_test, y_pred)

0.9491515437190667

In [37]:
f1_score(y_test, y_pred)

0.9611535402943868

In [38]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[16298   298]
 [ 2291 32029]]


In [34]:
false_positive_indices = sentence_test[(y_pred == 1) & (y_test == 0)] #не удалил, модель сказала удалить
false_negative_indices = sentence_test[(y_pred == 0) & (y_test == 1)] #удалил, а модель не удалила

In [36]:
false_positive_examples = pd.DataFrame(false_positive_indices, columns=['Values'])
false_negative_examples = pd.DataFrame(false_negative_indices, columns=['Values'])
print("False Positive Examples:")
print(false_positive_examples[:20])

print("\nFalse Negative Examples:")
print(false_negative_examples[:20])

False Positive Examples:
                                               Values
0   An abstract is not available for this content ...
1   We’ve built this digital research platform to ...
2   Dr. I. M. Nick enjoys a well-deserved reputati...
3   This study examines the syntactical difference...
4   THE ROOT STRUCTURE OF THE rìša TYPE VERBS Sum...
5   We’ve built this digital research platform to ...
6   We’ve built this digital research platform to ...
7   There exist systematic sound correspondences i...
8   Review article on Grammar of Mandarin . John B...
9                                      Editorial Note
10  Review of Yanez-Bouza, Nuria. Grammar, Rhetori...
11  We’ve built this digital research platform to ...
12  Book reviewed in this article:\n\n\n\nNoam Cho...
13  Preview this article: Response to Traugott, Pa...
14  With this Report (here the n° 2) we present re...
15  Article Klaus Zimmermann/Birte Kellermeier-Reh...
16                               Short communications
17 

In [37]:
false_positive_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])
false_negative_df = pd.DataFrame(columns=[ 'abstract', 'pred', 'true'])

# for example in false_positive_examples.values:
#     false_positive_df = false_positive_df.append({ 'abstract': example, 'pred': 1, 'true': 0}, ignore_index=True)
# for example in false_negative_examples.values:
#     false_negative_df = false_negative_df.append({ 'abstract': example, 'pred': 0, 'true': 1}, ignore_index=True)

In [42]:
false_positive_df.to_csv('false_positive_forest_modified.csv', index=False)
false_negative_df.to_csv('false_negative_forest_modified.csv', index=False)

Попробуем векторизировать через TF-IDF и поделить на два кластера через метод K-means

In [38]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_res2['abstract'])

k = 2  # Количество кластеров (0 и 1)
kmeans = KMeans(n_clusters=k)
kmeans.fit(tfidf_matrix)

df_res2['predicted_check'] = kmeans.labels_
df_res2['predicted_check'] = df_res2['predicted_check'].apply(lambda x: 1 if x == 0 else 0)

In [15]:
accuracy_score(df_res2['check'], df_res2['predicted_check'])

0.9080796137937536

In [40]:
false_positive = pd.DataFrame(columns=['abstract', 'check', 'predicted_check'])

# проходимся по строкам исходного датафрейма
# for index, row in df_res2.iterrows():
#     if row['predicted_check'] == 1 and row['check'] == 0:
#         false_positive = false_positive.append({'abstract': row['abstract'],
#                                                 'check': row['check'],
#                                                 'predicted_check': row['predicted_check']}, 
#                                                 ignore_index=True)
#len(false_positive)

In [41]:
true_positive = pd.DataFrame(columns=['abstract', 'check', 'predicted_check'])

# проходимся по строкам исходного датафрейма
# for index, row in df_res2.iterrows():
#     if row['predicted_check'] == 0 and row['check'] == 0:
#         true_positive = true_positive.append({'abstract': row['abstract'],
#                                                 'check': row['check'],
#                                                 'predicted_check': row['predicted_check']}, 
#                                                 ignore_index=True)
# len(true_positive)

In [42]:
false_positive.to_csv('false_positive_kmeans.csv', index=False)

In [43]:
false_negative = pd.DataFrame(columns=['abstract', 'check', 'predicted_check'])

# # проходимся по строкам исходного датафрейма
# for index, row in df_res2.iterrows():
#     if row['predicted_check'] == 0 and row['check'] == 1:
#         false_positive = false_positive.append({'abstract': row['abstract'],
#                                                 'check': row['check'],
#                                                 'predicted_check': row['predicted_check']}, 
#                                                 ignore_index=True)
# len(false_negative)

In [49]:
false_negative.to_csv('false_negative_kmeans.csv', index=False)

In [44]:
true_negative = pd.DataFrame(columns=['abstract', 'check', 'predicted_check'])

# # проходимся по строкам исходного датафрейма
# for index, row in df_res2.iterrows():
#     if row['predicted_check'] == 1 and row['check'] == 1:
#         true_negative = true_negative.append({'abstract': row['abstract'],
#                                                 'check': row['check'],
#                                                 'predicted_check': row['predicted_check']}, 
#                                                 ignore_index=True)
# len(true_negative)